## Video Load

In [13]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

from util import *

In [14]:
# 영상 FPS, 재생 속도, 프레임 스킵 수 설정
fps, speed, skip = 30, 1, 0

def getFrameDuration():
    return 1 / fps * (skip + 1)

# 영상 사이즈 기준 1920 x 1080
# 영상 크롭 및 사이즈 조정 (x1/4, x1/16)
class Frame:
    def __init__(self, frame, crop=64):
        self.x04 = cv2.resize(frame[crop:-crop], (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
        self.x16 = cv2.resize(self.x04,          (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)

        x04_hsv = cv2.split(cv2.cvtColor(self.x04, cv2.COLOR_BGR2HSV))
        x16_hsv = cv2.split(cv2.cvtColor(self.x16, cv2.COLOR_BGR2HSV))
        
        self.x04_H, self.x04_S, self.x04_V = x04_hsv
        self.x16_H, self.x16_S, self.x16_V = x16_hsv

In [15]:
class HistogramAnalysis:
    def __init__(self, frame):
        self.frame = frame

        # 히스토그램 분석
        self.hist = cv2.calcHist([frame], [0], None, [256], [0, 256])
        
        # 히스토그램 확률 밀도 함수
        self.hist = self.hist.squeeze() / self.hist.sum()

        # 히스토그램 평균
        self.mean = np.mean(self.hist * np.arange(256)) * 255

        # 히스토그램 분산 및 표준 편차
        var = self.hist * ((np.arange(256) - self.mean) ** 2)

        self.var = np.sum(var)

        self.std = (math.sqrt(np.sum(var[:int(self.mean)]) * 2), math.sqrt(np.sum(var[int(self.mean):]) * 2))

In [16]:
diff_graph, changed_graph = [0] * 256, [False] * 256

gap = 0

def detect_scene_change(prev : Frame, frame : Frame, hist_analyzed : HistogramAnalysis, debug=False):
    global gap

    # 이전 프레임과의 변화량 계산
    diff = int(np.sum(cv2.absdiff(prev.x16, frame.x16))) / frame.x16.size
    changed = abs(diff - diff_graph[-1]) > 10 * (1 + math.log(skip + 1) * 0.6) and gap >= 1
    
    diff_graph.append(diff) 
    diff_graph.pop(0)

    gap += getFrameDuration()
    
    if(changed): gap = 0

    # ===== 이하 디버깅을 위한 히스토그램 표시 =====
    if debug:
        changed_graph.pop(0) 
        changed_graph.append(changed)
        
        hist_img = get_histogram_image(np.array(diff_graph))

        hist_img[:, changed_graph] = (0, 0, 255)
        
        cv2.imshow("Scene Diffs", hist_img)
    # ===========================================

    return changed

In [17]:
def extract_field_feature(hist_analyzed : HistogramAnalysis, debug=False): 
    # 분석한 히스토그램을 바탕으로 필드 영역 추출
    l_weight, r_weight = 1.35, 0.5

    min_range, max_range = 24, 255

    std_l, std_r = hist_analyzed.std

    l, r = max(int(hist_analyzed.mean - std_l * l_weight), min_range), min(int(hist_analyzed.mean + std_r * r_weight), max_range)

    # ===== 이하 디버깅을 위한 히스토그램 표시 =====
    if debug:
        hist_img = get_histogram_image(hist_analyzed.hist)

        line_histogram_image(hist_img, l, (0, 0, 255))
        line_histogram_image(hist_img, r, (0, 0, 255))
        line_histogram_image(hist_img, int(hist_analyzed.mean), (255, 0, 255))

        cv2.imshow("Histogram Field", hist_img) 
    # ===========================================

    return cv2.inRange(hist_analyzed.frame, l, r)

def extract_mask(field_feature):
    # 필드로부터 플레이어 추출
    player_size = 12
    kernel = np.ones((player_size, player_size))
    field_feature = cv2.erode(field_feature, kernel) 

    player_mask = cv2.bitwise_not(field_feature)

    kernel = np.ones((9, 9))
    field_feature = cv2.dilate(field_feature, kernel)  

    kernel = np.ones((3, 3))
    field_feature = cv2.erode(field_feature, kernel)  

    # 필드 영역 마스크 생성
    field_mask = np.zeros(field_feature.shape, dtype=np.uint8)
    
    # 외곽선을 찾아서 필드 영역을 단순화
    contours, hierarchy = cv2.findContours(field_feature, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    contour_length = []

    for contour in contours:
        contour_length.append(cv2.arcLength(contour, True))

    if len(contour_length) > 0:
        contour = contours[int(np.argmax(contour_length))]

        cv2.drawContours(field_mask, [cv2.convexHull(contour)], -1, 255, -1) 

    # 필드로부터 플레이어 추출
    player_mask = cv2.copyTo(player_mask, field_mask)          

    return field_mask, player_mask

In [157]:
def extract_line(frame : Frame):
    line_feature = cv2.adaptiveThreshold(frame.x04_S, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 13)

    line_feature = cv2.copyTo(line_feature, frame.x04_field_mask) 

    # 직선 성분 검출
    lines = cv2.HoughLinesP(line_feature, 1, np.pi / 180, 120, minLineLength=50, maxLineGap=6)
    draw_lines(frame.x04, lines)
    cv2.imshow("Frame S", line_feature) 

In [124]:
def extract_lane_angle(frame : Frame):
    _, S, _ = frame.x16_hsv
    
    # 1차 Lane 특징 추출
    lane_feature = cv2.equalizeHist(S)

    kernel = np.array([[-1] * 10 + [+1] * 21 + [-1] * 10] * 5) / (10 * 5)
    lane_feature = cv2.filter2D(0   + lane_feature, -1, kernel)

    lane_feature = cv2.Blur(lane_feature, (7, 7), sigmaX=0, sigmaY=0)

    lane_feature = cv2.Canny(lane_feature, 12, 48) 

    # 정사각형으로 이미지 크롭
    height, width = lane_feature.shape

    scaler = (width - height) // 2
    
    lane_feature = lane_feature[:,scaler:-scaler]

    # 직선 성분 검출
    lines = cv2.HoughLinesP(lane_feature, 1, np.pi / 180, 120, minLineLength=50, maxLineGap=6)

    cv2.imshow("perspective", lane_feature)

    return lines

In [20]:
def analyze(prev : Frame, frame : Frame):
    analyzed_histogram = HistogramAnalysis(frame.x04_H)

    changed = detect_scene_change(prev, frame, analyzed_histogram, debug=True)
    
    frame.x04_field_mask, frame.x04_player_mask = extract_mask(extract_field_feature(analyzed_histogram, debug=True))

    extract_line(frame)

    # 원근 보정을 위해 필드에서 Lane의 특징 추출
    #lines = extract_lane_angle(cv2.copyTo(S, field_mask)
    
    cv2.imshow("mask", cv2.hconcat((frame.x04_field_mask, frame.x04_player_mask)))
                

In [21]:
def load_video(path):
    # Video Capture 객체 생성
    capture = cv2.VideoCapture(path)

    if capture.isOpened():
        prev, (run, frame) = None, capture.read()  # 다음 Frame 읽기
        
        if run: # Frame을 읽은 경우
            prev = frame = Frame(frame)

    current_skip = 0

    while capture.isOpened(): # Video Capture가 준비되었는지 확인
        run, frame = capture.read() # 다음 Frame 읽기
        
        if run: # Frame을 읽은 경우  
            if current_skip == 0:
                frame = Frame(frame)
                
                situation = analyze(prev, frame)

                cv2.imshow("Frame", frame.x04) 
                cv2.waitKey(max(int(1000 / (fps * speed)), 1)) # Millisecond 단위로 대기
            
                prev = frame
            
            current_skip = (current_skip + 1) % (skip + 1)
        else: # 재생이 완료되어 더 이상 Frame을 읽을 수 없는 경우
            break

    capture.release() # Capture 자원 반납 
    cv2.destroyAllWindows() # 창 제거

In [159]:
# Video가 저장된 경로 입력
PATH = r"video/soccer0.mp4"

# Video 재생 및 반환 (Numpy Array)
fps, speed, skip = 30, 2, 0
load_video(PATH)

KeyboardInterrupt: 

In [23]:
cv2.destroyAllWindows() # 창 제거

            #height, width, channel = frame.shape
            #cos = -int(math.cos(math.radians(angle)) * 1000)
            #sin = int(math.sin(math.radians(angle)) * 1000)
            
            #cv2.line(frame, (-cos+width//2, -sin+height//2), (cos+width//2, sin+height//2), (0, 0, 255), 1)
            
            #cv2.line(frame, (width // 2, height // 2), (width // 2, height // 2), (0, 0, 255), 5)
            #cv2.rectangle(frame, (width // 2 - height // 2, 0), (width // 2 + height // 2, height), (0, 255, 255), 10)
            #frame = cv2.copyTo(frame, field_mask)